### SageMaker Large Scale prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
pytorch_custom_image_name="ppi-extractor:cpu-1.0.0-202012310919"
instance_type = "ml.m5.large" 

In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

### Step 1: Convert pubtator format to inference json

The input pubtator files look like this.. These are converted to produce inference 

```text
20791654|a|Liver scan characteristics and liver function tests of 72 patients with proved hepatic malignancy (54 metastatic, 18 primary) were evaluated. Well-defined focal defects were observed in 83% of patients with metastatic and 77% of patients with primary liver carcinoma. In 10% of the patients with metastatic liver disease the distribution of radioactivity was normal. Four or more biochemical liver function tests were normal in 33% of metastatic and 29% of primary liver cancer patients. Hepatic enlargement was present in the scan in 94% of the patients with liver metastases; however, data obtained from 104 necropsies of patients with hepatic metastases showed that only 46% had hepatomegaly. We recommend, therefore, that a liver scan should be performed before major tumour surgery in every patient with known malignancy regardless of normal liver size or normal liver function tests.
20791654	58	66	patients	Species	9606
20791654	193	201	patients	Species	9606
20791654	229	237	patients	Species	9606
20791654	282	290	patients	Species	9606
20791654	478	486	patients	Species	9606
20791654	546	554	patients	Species	9606
20791654	624	632	patients	Species	9606
20791654	796	803	patient	Species	9606

20791817|a|5-Aminosalicylic acid given to rats as a single intravenous injection led to necrosis of the proximal convoluted tubules and of the renal papilla. These two lesions developed at the same time and the cortical lesions did not appear to be a consequence of the renal papillary necrosis. Since the compound possesses the molecular structure both of a phenacetin derivative and of a salicylate these observations may be relevant to the problem of renal damage incident to abuse of analgesic compounds and suggest the possibility that in this syndrome cortical lesions may develop independently of renal papillary necrosis.
20791817	31	35	rats	Species	10116

```

In [4]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [5]:
s3_input_pubtator = "s3://aegovan-data/pubmed_json_parts_annotation_iseries/"
s3_id_mapping_file="s3://aegovan-data/settings/HUMAN_9606_idmapping.dat"

s3_output_pubmed_asbtract = f"s3://aegovan-data/pubmed_asbtract/inference_multi_{date_fmt}/"

In [6]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python3"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=10,
                                       max_runtime_in_seconds=172800,
                                       volume_size_in_gb = 50,
                                       network_config=NetworkConfig(enable_network_isolation=False)
                                       )


sm_local_input_pubtator_txt = "/opt/ml/processing/input/data/json"
sm_local_input_idmapping = "/opt/ml/processing/input/data/mapping"
sm_local_output = "/opt/ml/processing/output"


script_processor.run(
        code='source/datatransformer/pubtator_annotations_inference_transformer.py',

        arguments=[
        
            sm_local_input_pubtator_txt,
            sm_local_output,
           "{}/{}".format(sm_local_input_idmapping,s3_id_mapping_file.split("/")[-1]) 

        ],
    
       inputs=[
                ProcessingInput(
                    source=s3_input_pubtator,
                    destination=sm_local_input_pubtator_txt,
                    s3_data_distribution_type="ShardedByS3Key")

            ,ProcessingInput(
                    source=s3_id_mapping_file,
                    destination=sm_local_input_idmapping,
                    s3_data_distribution_type="FullyReplicated")
            ],

        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_pubmed_asbtract,
                output_name='inferenceabstracts')]
    )

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  ppi-extractor-2020-12-31-10-19-17-793
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://aegovan-data/pubmed_json_parts_annotation_iseries/', 'LocalPath': '/opt/ml/processing/input/data/json', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'S3Input': {'S3Uri': 's3://aegovan-data/settings/HUMAN_9606_idmapping.dat', 'LocalPath': '/opt/ml/processing/input/data/mapping', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-324346001917/ppi-extractor-2020-12-31-10-19-17-793/input/code/pubtator_annotations_inference_transformer.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'inf

2020-12-31 10:24:02,143 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0866.json.txt with records 14959
2020-12-31 10:24:02,173 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0404.json.txt with records 4603
2020-12-31 10:24:02,920 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0200.json.txt with records 9
2020-12-31 10:24:02,972 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0230.json.txt with records 261
2020-12-31 10:24:03,391 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0427.json.txt with records 1
2020-12-31 10:24:03,793 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0644.json.txt with records 16204
2020-12-31 10:24:03,581 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0530.json.txt with records 17294
2020-12-31 10:24:03,851 - __main__ - INFO - Processed file 

2020-12-31 10:24:17,130 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0562.json.txt with records 19710
2020-12-31 10:24:17,122 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0145.json.txt with records 230
2020-12-31 10:24:17,652 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0275.json.txt with records 7613
2020-12-31 10:24:18,242 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0951.json.txt with records 12551
2020-12-31 10:24:18,256 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0800.json.txt with records 16257
2020-12-31 10:24:17,767 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0659.json.txt with records 16974
2020-12-31 10:24:18,935 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0241.json.txt with records 437
2020-12-31 10:24:18,944 - __main__ - INFO - Processed

2020-12-31 10:24:39,212 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0940.json.txt with records 12430
2020-12-31 10:24:39,106 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0358.json.txt with records 13948
2020-12-31 10:24:39,289 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0334.json.txt with records 13833
2020-12-31 10:24:40,539 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0003.json.txt with records 1546
2020-12-31 10:24:40,014 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0052.json.txt with records 4506
2020-12-31 10:24:40,032 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0796.json.txt with records 15456
2020-12-31 10:24:41,077 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0577.json.txt with records 5922
2020-12-31 10:24:41,327 - __main__ - INFO - Process

2020-12-31 10:24:58,477 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0630.json.txt with records 15353
2020-12-31 10:24:58,973 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0308.json.txt with records 12621
2020-12-31 10:24:59,527 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0698.json.txt with records 16753
2020-12-31 10:24:59,221 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0426.json.txt with records 2
2020-12-31 10:24:59,836 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0817.json.txt with records 16077
2020-12-31 10:24:59,962 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0589.json.txt with records 17136
2020-12-31 10:25:00,685 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0422.json.txt with records 18122
2020-12-31 10:25:00,306 - __main__ - INFO - Processe

2020-12-31 10:25:19,928 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0195.json.txt with records 9
2020-12-31 10:25:20,052 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0781.json.txt with records 14453
2020-12-31 10:25:19,679 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0068.json.txt with records 3171
2020-12-31 10:25:20,256 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0125.json.txt with records 1056
2020-12-31 10:25:20,495 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0035.json.txt with records 186
2020-12-31 10:25:20,610 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0429.json.txt with records 356
2020-12-31 10:25:21,108 - __main__ - INFO - Processed file /opt/ml/processing/input/data/json/pubmed19n0900.json.txt with records 8248
2020-12-31 10:25:20,844 - __main__ - INFO - Processed file 

KeyboardInterrupt: 

## Step 2: Run predictions

In [28]:
jobs = [
"ppi-bert-2020-12-28-06-14-27-510",
"ppi-bert-2020-12-28-06-13-28-613",
"ppi-bert-2020-12-28-06-12-43-937",
"ppi-bert-2020-12-28-06-11-48-471",
"ppi-bert-2020-12-28-06-10-53-005",
"ppi-bert-2020-12-28-06-10-00-183",
"ppi-bert-2020-12-28-06-09-00-491",
"ppi-bert-2020-12-28-06-08-02-139",
"ppi-bert-2020-12-28-06-07-01-234",
"ppi-bert-2020-12-28-06-06-07-198"
]

s3_model_path_format = "s3://aegovan-data/results/{}/output/model.tar.gz"

s3_model_paths = [s3_model_path_format.format(j) for j in jobs]

In [29]:
s3_output_ensemble_models= "s3://aegovan-data/ensemble_models/{}".format("2020-12-28-06-part")

In [ ]:
# python ${source_dir}/algorithms/main_predict.py ${network_type}  ${src_local_file}  ${network_local_path}  ${dest_local_path}  --positives-filter-threshold ${threshold}


# python ${source_dir}/algorithms/ensemble_inference_batchs3.py --input-comma-sep-path  ${network_s3_csv} --dest-dir ${network_local_path}
# python ${source_dir}/algorithms/main_predict.py PpiMulticlassDatasetFactory  ${datajson}  ${artefactsdir}   ${outdir}  


### Prepare ensemble models
TODO: This is just a hack to untar a bunch of zipped models and upload them to a single s3 locaton. Have a single processing job to this this is an overkill...

In [ ]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python3"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=1,
                                       max_runtime_in_seconds=172800,
                                       volume_size_in_gb = 50,
                                       network_config=NetworkConfig(enable_network_isolation=False)
                                       )


sm_local_input = "/opt/ml/processing/input/models"
sm_local_output = "/opt/ml/processing/output"



def get_processing_inputs_s3_local_path(s3_model_paths, sm_local_input):
    # Map the s3 model path to local input path
    inputs = []
    for i, s3_path in enumerate(s3_model_paths):
         p = ProcessingInput(
                        source=s3_path,
                        destination="{}/{}".format(sm_local_input.rstrip("/"), i)
         )
         inputs.append(p)
    return inputs


# Work around to get over the processing job input limit size
chunk_size=5
for i in range(0, len(s3_model_paths), chunk_size ):
    
    script_processor.run(
            code='source/algorithms/ensemble_inference_prepare_models.py',

            arguments=[
                "--input-dir",
                sm_local_input,
                "--dest-dir",
                sm_local_output

            ],

            inputs=get_processing_inputs_s3_local_path(s3_model_paths[i:i+chunk_size], sm_local_input),


            outputs=[ProcessingOutput(
                    source=sm_local_output, 
                    destination=s3_output_ensemble_models,
                    output_name='models')]
        )





Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  ppi-extractor-2020-12-31-11-39-02-843
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://aegovan-data/results/ppi-bert-2020-12-28-06-14-27-510/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/models/0', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'S3Input': {'S3Uri': 's3://aegovan-data/results/ppi-bert-2020-12-28-06-13-28-613/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/models/1', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'S3Input': {'S3Uri': 's3://aegovan-data/results/ppi-bert-2020-12-28-06-12-43-937/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/models/2', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-4', 'S3Inp

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  ppi-extractor-2020-12-31-11-44-38-995
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://aegovan-data/results/ppi-bert-2020-12-28-06-10-00-183/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/models/0', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'S3Input': {'S3Uri': 's3://aegovan-data/results/ppi-bert-2020-12-28-06-09-00-491/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/models/1', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'S3Input': {'S3Uri': 's3://aegovan-data/results/ppi-bert-2020-12-28-06-08-02-139/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/models/2', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-4', 'S3Inp